In [ ]:
from gensim.models import KeyedVectors
from tqdm import tqdm
import numpy as np

In [ ]:
glove = '/home/manni/embs/gloveW2V.6B.300d.txt'
w2v = '/home/manni/embs/w2v.bin.gz'
wiki2v = '/home/manni/embs/enwiki_20180420_300d.txt.bz2'
dconf = 

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(glove, binary=False)

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=True)

In [ ]:
wiki_model = KeyedVectors.load_word2vec_format(wiki2v, binary=False)

In [ ]:
words = wiki_model.most_similar('bank',topn=100)

In [ ]:
wiki_model.most_similar('bank',topn=100)

In [ ]:
#w2v_model.vocab

In [ ]:
def total_overlap(word,model,topn=100):
    overlap_matrix = np.zeros((topn,topn),dtype=np.int8)
    words = model.most_similar(word,topn=topn)
    words = set([i[0] for i in words])
    nns = dict()
    for word in words:
        _words = model.most_similar(word,topn=5)
        _words = set([i[0] for i in _words])
        nns[word]=_words
    for i,(k,a) in enumerate(nns.items()):
        for j,(kk,b) in enumerate(nns.items()):
            if k == kk:
                overlap_matrix[i][j]=1
                continue
            overlap = len(a & b)
            overlap_matrix[i][j]=overlap
    return nns,overlap_matrix

In [ ]:
nns,ovm = total_overlap('bank',glove_model,topn=100)

In [ ]:
def print_closer(n):
    print(voc[n])
    print(':::::')
    nn = set()
    for i,val in enumerate(ovm[n]>0):
        if val:
            print(i,voc[i])
            nn.add(voc[i])
    return nn

In [ ]:
a = print_closer(3)
b = print_closer(15)
print(a&b)

In [ ]:
clusters = dict()
voc = list(nns.keys())
for v in voc:
    clusters[v]=-1
c = 0
for i in range(len(ovm)):
    for j in range(len(ovm)):
        if i == j:
            continue
        vec = ovm[i]
        vec = vec>0
        vec_ = ovm[j]
        vec_= vec_>0
        list_a = [voc[i_] for i_,x in enumerate(vec) if x]
        list_b = [voc[i_] for i_,x in enumerate(vec_) if x]
        overlap = len(set(list_a) & set(list_b))
        if overlap>3:
            #print(i,':',voc[i],j,':',voc[j])
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
            if clusters[voc[i]]>-1 and clusters[voc[j]]>-1:
                c_ = min(clusters[voc[i]],clusters[voc[j]])
                clusters[voc[i]]=c_
                clusters[voc[j]]=c_
            elif clusters[voc[i]]>-1:
                clusters[voc[j]]=clusters[voc[i]]
            elif clusters[voc[j]]>-1:
                clusters[voc[i]]=clusters[voc[j]]
            else:
                clusters[voc[i]]=c
                clusters[voc[j]]=c
                c+=1
            #if i == 10 or j == 10:
            #    print(i,j,clusters[voc[i]],clusters[voc[j]])
        #if np.array_equal(vec,vec_):
        #    print(i,':',voc[i],j,':',voc[j])
    #print('***')

In [ ]:
clusters

In [ ]:
clusters_ = dict()
for i,c in enumerate(clusters.values()):
    if c == -1:
        continue
    if c in clusters_:
        clusters_[c].append(voc[i])
    else:
        clusters_[c] = [voc[i]]

In [ ]:
clusters_

In [ ]:
def get_matrix(model,vocab):
    matrix = list()
    for word in vocab:
        vec = model.get_vector(word)
        matrix.append(vec)
    return matrix

In [ ]:
matrix = get_matrix(glove_model,vocab)

In [ ]:
matrix = np.asarray(matrix)

# validate embeddings

# clustering

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

In [ ]:
db = DBSCAN(eps=0.4,min_samples=2,metric='cosine').fit(matrix)

In [ ]:
labels = db.labels_
clusters = dict()
for i,c in enumerate(labels):
    if c == -1:
        continue
    if c in clusters:
        clusters[c].append(vocab[i])
    else:
        clusters[c] = [vocab[i]]

In [ ]:
clusters

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
X = [[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]]
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
clustering.labels_

# wordnet

In [6]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
from collections import Counter

In [7]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [ ]:
wn.synsets('dog')[0].hyponyms()

In [ ]:
wn.synset('dog.n.01').definition()

In [ ]:
wn.synset('frump.n.01').definition()

In [ ]:
wn.synset('dog.n.03').definition()

In [ ]:
dog = wn.synset('dog.n.01')
dog.hypernyms()

In [ ]:
dog.hyponyms()

In [ ]:
wn.synsets('bank')[9].hyponyms()

In [8]:
vocab = list(wn.all_synsets())

In [9]:
len(vocab)

117659

In [10]:
vocab_ = set()
for ss in vocab:
    word = ss.name().split('.')[0]
    if '_' in word:
        continue
    vocab_.add(word)

In [11]:
len(vocab_)

58736

In [12]:
list(vocab_)[:10]

['',
 'incorporation',
 'labor-intensive',
 'pulasan',
 'ailurophobia',
 'animal',
 'skeet',
 'tranquillity',
 'properly',
 'crawlspace']

In [13]:
def get_uniques(input_list):
    uniques = set()
    for k,v in Counter(input_list).items():
        if v>1:
            continue
        uniques.add(k)
    return uniques

In [14]:
lemmatizer = WordNetLemmatizer()
#ps = PorterStemmer()
stops = set(stopwords.words('english'))
word_desc = dict()
for _word in vocab_:
    word = lemmatizer.lemmatize(_word)
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        txt = re.sub(r'\W+', ' ', txt)
        words = [w for w in txt.split()]
        #txts = synset.examples()
        #txt = ' '.join(txts)
        #txt = re.sub(r'\W+', ' ', txt)
        #words_ = [w for w in txt.split()]
        #words = words + words_
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        _words = words+hypo_words+hyper_words
        words = list()
        for w in _words:
            words.extend(w.split('_'))
        words = [lemmatizer.lemmatize(w) for w in words if w not in stops]
        words = set(words)
        #add to dict
        if word in word_desc:
            word_desc[word].append(words)
        else:
            word_desc[word]=[words]

In [15]:
len(word_desc['dog'])

8

In [16]:
word2desc = dict()
for k,v in word_desc.items():
    word2desc[k] = list()
    all_words = [item for sublist in v for item in sublist]
    uniques = get_uniques(all_words)
    for words in v:
        words = set(words)
        word2desc[k].append(words&uniques)

In [22]:
word2desc['dogging']

[{'catch', 'go', 'hound', 'intent', 'pursue', 'quest', 'run', 'tree'},
 {'indefatigable', 'pursuit', 'relentless'}]

In [19]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [20]:
sv.save(word2desc,"word2desc")

Saved the data


In [31]:
word2desc['anarchism'][0]

{'abolition', 'favoring', 'government', 'orientation', 'political', 'theory'}

In [ ]:
print(word2desc['fast'])
print(len(word2desc['fast']))

In [24]:
def verbose(word):
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        txt = re.sub(r'\W+', ' ', txt)
        print(txt)
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        print(hyper_words)
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        print(hypo_words)
        print('______')

In [ ]:
lemmatizer.lemmatize("faster")

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
doc = nlp("a part of a river where the current is very fast")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

a a DET DT det x True True
part part NOUN NN ROOT xxxx True True
of of ADP IN prep xx True True
a a DET DT det x True True
river river NOUN NN pobj xxxx True False
where where SCONJ WRB advmod xxxx True True
the the DET DT det xxx True True
current current NOUN NN nsubj xxxx True False
is be AUX VBZ relcl xx True True
very very ADV RB advmod xxxx True True
fast fast ADV RB acomp xxxx True False


In [25]:
verbose('dogging')

go after with the intent to catch
['pursue']
['hound', 'quest', 'run_down', 'tree']
______
relentless and indefatigable in pursuit or as if in pursuit
[]
[]
______


In [ ]:
verbose('rapid')

In [ ]:
verbose('fast')

In [ ]:
word2desc['rapid']

In [ ]:
word2desc['fast']

In [ ]:
def get_overlap(a,b):
    a = word2desc[a]
    b = word2desc[b]
    overlap = 0
    match = None
    for i,x in enumerate(a):
        for j,y in enumerate(b):
            if len(x & y) > overlap:
                overlap = len(x & y)
                match = (i,j)
    return match, overlap

In [ ]:
get_overlap('fast','rapid')

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
print(ps.stem('moving'))
print(lemmatizer.lemmatize('moving'))

In [ ]:
word2desc['authority'][4]

In [ ]:
len(vocab_)

In [ ]:
list(vocab_)[:10]

In [ ]:
dog.examples()

In [ ]:
for i in vocab_[100:110]:
    print(i)
    txt = i.definition().lower()
    print(txt)
    txt = re.sub(r'\W+', ' ', txt)
    print(txt)
    words = [word for word in txt.split() if word not in stops]
    print(set(words))
    word = i.name().split('.')[0]
    print(wn.synsets(word))
    print()